In [1]:
import os, torch 
import numpy as np
from monai import transforms
from torch.utils.data import DataLoader
from dataset.prostateDataset import prostateDataset
from models.resnet import generate_model

In [11]:
def _get_data_loader(modality):
    file_root = "/Users/qixinhu/Project/CUHK/Prostate/PAIsData/0426/qixin/CLS"
    file_list = sorted(os.listdir(os.path.join(file_root, modality)))
            
    test_images = []
    for sample in file_list:
        test_images.append(os.path.join(file_root, modality, sample))
    test_img_transform = transforms.Compose(
        [
            transforms.EnsureChannelFirst(channel_dim="no_channel"),
        ]
    )
    test_ds = prostateDataset(npz_files=test_images, img_transforms=test_img_transform, seg_transforms=None)
    test_loader = DataLoader(test_ds, batch_size=1, shuffle=False,num_workers=0, pin_memory=True)
    return test_loader

n_classes = 2
model_depth = 18
modality = 'ALL'
model = generate_model(model_depth=model_depth,
                    input_H=128, input_W=128, input_D=36, n_classes=n_classes)
model.load_state_dict(torch.load(f"runs/{modality}/resnet18/model_best.pt", map_location='cpu'))
model.eval()
check_loder = _get_data_loader(modality)
check_batch = next(iter(check_loder))

/Users/qixinhu/Project/CUHK/Prostate/ProstateInitialSub/classification/models/resnet.py:173: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [12]:
with torch.no_grad():
    check_image = check_batch['image']
    check_pred  = model(check_image)